Import stdPackages manually:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os, pandas as pd, numpy as np, pickle, math, pyDbs, scipy
from scipy import optimize, sparse, stats
from importlib.metadata import version
from IPython.display import IFrame
_repo = 'EnergyEconGroupWork'
d = {'main': os.path.join(os.getcwd().split(_repo,1)[0],_repo),
     'curr': os.getcwd()}
d.update({'py': os.path.join(d['main'], 'py'),
          'data': os.path.join(d['main'], 'Data')})
os.chdir(d['py'])
import base, lpCompiler, lpModels, mBasicPH_storage
os.chdir(d['curr'])

For working with the model, we first import some packages and the modelclass:

In [1]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/paul/EnergyEcon/EnergyEconGroupWork


In [2]:
%run ./stdPackages.ipynb
from pyDbs import *

# Windows
#os.chdir(os.path.join(d['curr'],'EnergyEconGroupWork', 'py'))
# MACOS
os.chdir(os.path.join(d['curr'], 'py'))

from mBasicPH_storage import *
os.chdir(d['curr'])

Next we load the database:

In [3]:
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'GeneratorsVariables','StorageVariables','TransmissionLines'],
        'maps': ['LoadMaps','GeneratorsMaps','StorageMaps', 'MarketMaps'],
        'variable2D': ['HourlyVariation']}
# Windows
#path_to_data = os.path.join(d['curr'],'EnergyEconGroupWork', 'Data', 'mBasicPH_storage_Data.xlsx')
# MACOS
path_to_data = os.path.join(d['curr'], 'Data', 'mBasicPH_storage_Data.xlsx')

db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

Convert variables to floats:

In [ ]:
[db.__setitem__(var,db[var].astype(float).sort_index()) for var in db.getTypes('variable').keys()];

We need to read in heat areas manually:

In [ ]:
db['g_H'] = db['g_E2g_H'].get_level_values('g_H')

Collect all areas in one:

In [ ]:
db['g'] = db['g_E'].copy()

Finally, we will assume that linelosses and trade are zero such that the marginal willingness to pay and marginal generation costs of "transmission lines" does not have to be corrected. 

We then initialize and solve the model:

In [ ]:
m = mSimple(db)
m()